# Data Preparation

## Import Libraries

In [14]:
import pandas as pd 
import numpy as np 
import scipy as sp 

import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession

# Importar funciones necesarias
from pyspark.sql.functions import col, to_date, weekofyear,year, month, dayofmonth, sum
from pyspark.sql.functions import col, sum as spark_sum
from pyspark.sql.functions import regexp_replace, col, when
from pyspark.sql.functions import format_number
from pyspark.sql.types import IntegerType,FloatType
# Puedes obtener estadísticas específicas para una columna
from pyspark.sql.functions import mean, min, max
from pyspark.sql.functions import approx_count_distinct
from pyspark.sql.window import Window
from pyspark.sql.functions import log1p
from pyspark.sql import functions as F


# Import Data

This dataset contains the full LendingClub data available from [their site](https://www.lendingclub.com). There are separate files for accepted and rejected loans. The accepted loans also include the FICO scores, which can only be downloaded when you are signed in to LendingClub and download the data (https://www.kaggle.com/datasets/wordsforthewise/lending-club).



In [2]:

# Crear la sesión de Spark
spark = SparkSession.builder.appName("AppLending").getOrCreate()

# Rutas de los archivos CSV comprimidos
accepted_r = 'hdfs://namenode:8020/datasets/raw/accepted_2007_to_2018Q4.csv.gz'
rejected_r = 'hdfs://namenode:8020/datasets/raw/rejected_2007_to_2018Q4.csv.gz'
#accepted_df = spark.read.csv("hdfs://namenode:8020/datasets/raw/accepted_2007_to_2018Q4.csv.gz", header=True, inferSchema=True)

# Leer los archivos CSV comprimidos usando PySpark
accepted_df = spark.read.option("header", "true").option("inferSchema", "true").csv(accepted_r)
rejected_df = spark.read.option("header", "true").option("inferSchema", "true").csv(rejected_r)

In [3]:
# Haciendo una copia del DataFrame usando alias
accepted_dfm = accepted_df.alias("accepted_dfm")
rejected_dfm = rejected_df.alias("rejected_dfm")

## Explore Data

In [7]:
print((rejected_dfm.count(), len(rejected_dfm.columns)))

(27648741, 9)


In [8]:
print((accepted_dfm.count(), len(accepted_dfm.columns)))

(2260701, 151)


In [9]:
rejected_dfm.printSchema()

root
 |-- Amount Requested: double (nullable = true)
 |-- Application Date: date (nullable = true)
 |-- Loan Title: string (nullable = true)
 |-- Risk_Score: string (nullable = true)
 |-- Debt-To-Income Ratio: string (nullable = true)
 |-- Zip Code: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Employment Length: string (nullable = true)
 |-- Policy Code: string (nullable = true)



In [9]:
rejected_dfm.show()

+----------------+----------------+--------------------+----------+--------------------+--------+-----+-----------------+-----------+
|Amount Requested|Application Date|          Loan Title|Risk_Score|Debt-To-Income Ratio|Zip Code|State|Employment Length|Policy Code|
+----------------+----------------+--------------------+----------+--------------------+--------+-----+-----------------+-----------+
|          1000.0|      2007-05-26|Wedding Covered b...|     693.0|                 10%|   481xx|   NM|          4 years|        0.0|
|          1000.0|      2007-05-26|  Consolidating Debt|     703.0|                 10%|   010xx|   MA|         < 1 year|        0.0|
|         11000.0|      2007-05-27|Want to consolida...|     715.0|                 10%|   212xx|   MD|           1 year|        0.0|
|          6000.0|      2007-05-27|             waksman|     698.0|              38.64%|   017xx|   MA|         < 1 year|        0.0|
|          1500.0|      2007-05-27|              mdrigo|     5

In [4]:
# Aplicar tratamiento a los caracteres no numéricos y convertir a tipo Integer
# Definir una función UDF para convertir a tipo Float
def convert_to_float(column):
    return when(col(column).cast("float").isNull(), 0).otherwise(col(column).cast("float"))

# Aplicar tratamiento a los caracteres no numéricos y convertir a tipo Float
rejected_dfm = rejected_dfm.withColumn("Risk_Score", convert_to_float("Risk_Score"))
rejected_dfm = rejected_dfm.withColumn("Debt-To-Income Ratio", convert_to_float("Debt-To-Income Ratio"))
rejected_dfm = rejected_dfm.withColumn("Zip Code", regexp_replace(col("Zip Code"), "[^0-9]", "").cast("string"))
rejected_dfm = rejected_dfm.withColumn("Policy Code", convert_to_float("Policy Code"))

In [11]:
# Verificar valores nulos por columna y contarlos
null_counts = rejected_dfm.select([spark_sum(col(c).isNull().cast("int")).alias(c) for c in rejected_dfm.columns])
null_counts.show()

+----------------+----------------+----------+----------+--------------------+--------+-----+-----------------+-----------+
|Amount Requested|Application Date|Loan Title|Risk_Score|Debt-To-Income Ratio|Zip Code|State|Employment Length|Policy Code|
+----------------+----------------+----------+----------+--------------------+--------+-----+-----------------+-----------+
|               0|               0|      1303|  18497631|                   1|     293|   22|           951355|        919|
+----------------+----------------+----------+----------+--------------------+--------+-----+-----------------+-----------+



| Campo                | Descripción                                                                                                     |
|----------------------|-----------------------------------------------------------------------------------------------------------------|
| Amount Requested     | La cantidad total solicitada por el prestatario.                                                                  |
| Application Date     | La fecha en la cual el prestatario presentó la solicitud.                                                         |
| Loan Title           | El título del préstamo proporcionado por el prestatario.                                                            |
| Risk_Score           | Para aplicaciones anteriores al 5 de noviembre de 2007, la puntuación de riesgo es la puntuación FICO del prestatario. Para aplicaciones después del 5 de noviembre de 2018, la puntuación de riesgo es la puntuación Vantage del prestatario. |
| Debt-To-Income Ratio | Una proporción calculada utilizando los pagos mensuales totales del prestatario en las obligaciones de deuda totales, excluyendo la hipoteca y el préstamo solicitado a través de LC, dividido por los ingresos mensuales autodeclarados del prestatario. |
| Zip Code             | Los primeros 3 números del código postal proporcionado por el prestatario en la solicitud de préstamo.           |
| State                | El estado proporcionado por el prestatario en la solicitud de préstamo.                                            |
| Employment Length    | Duración del empleo en años. Los valores posibles están entre 0 y 10, donde 0 significa menos de un año y 10 significa diez o más años. |
| Policy Code          | Código de política disponible públicamente. policy_code=1 para nuevos productos no disponibles públicamente, policy_code=2. |


In [12]:
# Revisemos el Target del los Créditos Rechazados
rejected_dfm_keys = rejected_dfm.select("Risk_Score").distinct()
rejected_dfm_keys.show()

+----------+
|Risk_Score|
+----------+
|     714.0|
|     550.0|
|     769.0|
|     364.0|
|     886.0|
|     729.0|
|     803.0|
|     362.0|
|     425.0|
|     955.0|
|     332.0|
|     692.0|
|     797.0|
|     911.0|
|     615.0|
|     829.0|
|     331.0|
|     308.0|
|     500.0|
|     547.0|
+----------+
only showing top 20 rows



In [26]:
# Obtener solo las columnas numéricas
rejected_dfm_numeric= [col_name for col_name, col_type in rejected_dfm.dtypes if col_type in ['int', 'bigint', 'double', 'float']]
# Estadísticas descriptivas de las columnas numéricas

# Calcular estadísticas descriptivas solo para las columnas numéricas
# rejected_dfm.select(rejected_dfm_numeric).describe().show()

# Calcular estadísticas descriptivas solo para las columnas numéricas
descriptive_stats= rejected_dfm.select(rejected_dfm_numeric).describe()

# Aplicar formato a las columnas numéricas para mostrar solo dos decimales
rejected_dfm_stats = descriptive_stats.select(
    descriptive_stats["summary"],
    *[format_number(descriptive_stats[col].cast("double"), 2).alias(col) for col in descriptive_stats.columns[1:]]
)

# Mostrar la tabla de estadísticas descriptivas formateada
rejected_dfm_stats.show()

+-------+----------------+------------+--------------------+-------------+
|summary|Amount Requested|  Risk_Score|Debt-To-Income Ratio|  Policy Code|
+-------+----------------+------------+--------------------+-------------+
|  count|   27,648,741.00|9,151,110.00|       27,648,740.00|27,647,822.00|
|   mean|       13,133.24|      628.17|           35,218.37|         0.64|
| stddev|       15,009.64|       89.94|          547,230.76|        11.27|
|    min|            0.00|        0.00|                0.00|         0.00|
|    max|    1,400,000.00|      990.00|      500,000,320.00|       200.00|
+-------+----------------+------------+--------------------+-------------+



***Interpretación***:<br>
La ***cantidad total solicitada*** por los prestatarios varía desde 0 hasta 1,400,000, con una media de aproximadamente 13,133.24.
Risk Score (Puntuación de Riesgo):

Las ***puntuaciones de riesgo*** van desde 0 hasta 990. La media es aproximadamente 628.17.
Debt-To-Income Ratio (Relación Deuda-Ingresos):

La ***relación deuda-ingresos*** varía significativamente, con una media de alrededor de 35,218.37. Sin embargo, hay una gran desviación estándar, indicando una variabilidad significativa.
Policy Code (Código de Política):

El ***código de política*** tiene dos valores posibles: 1 y 2, y parece tener una media de alrededor de 0.64, con una desviación estándar de 11.27.

#### Hipostesis planteada
***Relación entre Amount Requested y Debt-To-Income Ratio:***

Hipótesis: Existe una correlación significativa entre la cantidad solicitada y la relación deuda-ingresos. Los prestatarios que solicitan grandes cantidades podrían tener relaciones deuda-ingresos más altas.
***Variación de Risk Score por State:***

Hipótesis: La puntuación de riesgo puede variar según el estado del prestatario. Algunos estados podrían tener prestatarios con puntuaciones de riesgo más altas o más bajas en promedio.

***Cambios en la Amount Requested a lo largo del tiempo:***
Hipótesis: La cantidad solicitada podría haber experimentado cambios a lo largo del tiempo. Podría haber tendencias o patrones notables en la cantidad solicitada a lo largo de los años.

***Impacto de Employment Length en la Risk Score:***
Hipótesis: La duración del empleo podría estar relacionada con la puntuación de riesgo. Por ejemplo, prestatarios con empleos a largo plazo podrían tener puntuaciones de riesgo más bajas.

***Diferencias en Amount Requested entre Policy Code 1 y Policy Code 2:***
Hipótesis: Podría haber diferencias significativas en la cantidad solicitada entre los productos nuevos no disponibles públicamente (policy_code=1) y los productos públicamente disponibles (policy_code=2).

In [7]:
rejected_dfm.show()

+----------------+----------------+--------------------+----------+--------------------+--------+-----+-----------------+-----------+
|Amount Requested|Application Date|          Loan Title|Risk_Score|Debt-To-Income Ratio|Zip Code|State|Employment Length|Policy Code|
+----------------+----------------+--------------------+----------+--------------------+--------+-----+-----------------+-----------+
|          1000.0|      2007-05-26|Wedding Covered b...|     693.0|                 0.0|     481|   NM|          4 years|        0.0|
|          1000.0|      2007-05-26|  Consolidating Debt|     703.0|                 0.0|     010|   MA|         < 1 year|        0.0|
|         11000.0|      2007-05-27|Want to consolida...|     715.0|                 0.0|     212|   MD|           1 year|        0.0|
|          6000.0|      2007-05-27|             waksman|     698.0|                 0.0|     017|   MA|         < 1 year|        0.0|
|          1500.0|      2007-05-27|              mdrigo|     5